# Class 7 - Regularization

## Regularized loss minimization

*Regularized loss minimization (RLM)* is a learning rule in which we minimize the empirical risk together with a regularization function $\rho : \mathbb{R}^d \to \mathbb{R}$, which often leads to slightly higher bias but significantly reduced variance in the bias-variance tradeoff.

If the hypothesis class is parametrized by $ \mathbf{w} \in \mathbb{R}^d$, then the regularized loss minimization rule outputs a hypothesis in 

$$\arg \min_{\mathbf{w} \in \mathbb{R}^d} \big(\hat{R}(\mathbf{w}) + \rho(\mathbf{w})\big).$$

The regularization function $\rho$ is typically chosen to penalize how complex the hypothesis is, in order to prevent overfitting. We will consider the most widely used types, the L1 and L2 regularization.

## L2 regularization

Taking $\rho(\mathbf{w}) = \lambda \| \mathbf{w} \|_2^2$, where $\lambda > 0$ is a scalar and the norm is $ \| \mathbf{w} \|_2 = \big(\sum_{j=1}^{d} (w^{(j)})^2\big)^{1/2}$, we have the learning rule 

$$\arg \min_{\mathbf{w} \in \mathbb{R}^d} \big(\hat{R}(\mathbf{w}) + \lambda \| \mathbf{w} \|_2^2 \big),$$

called *L2* (or *Tikhonov*) *regularization*. An equivalent way to write this problem is 

$$\arg \min_{\mathbf{w} \in \mathbb{R}^d} \hat{R}(\mathbf{w}) \quad \text{subject to} \ \sum_{j=1}^{d} \big(w^{(j)}\big)^2 \leq C,$$

which makes explicit the size constraint on the parameters.

### Ridge regression

Applying the RLM with L2 regularization to linear regression with the squared loss, called *ridge regression*, we obtain the learning rule

$$ \arg \min_{β \in \mathbb{R}^d} \bigg( \lambda \| β \|_2^2  + \frac{1}{n} \sum_{i=1}^{n} \big(y_i - \langle β , x_i \rangle \big)^2 \bigg).$$ 

To solve the minimization problem, we set the gradient equal to zero and obtain 

$$ (\lambda n I + X^T X) β = X^T y.$$

The matrix $\lambda n I + X^T X$ is invertible since it is positive definite for $\lambda > 0$, and we have the solution

$$ β =(\lambda n I + X^T X)^{-1}X^T y.$$

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets, model_selection, linear_model, metrics 

def L2_regularized_risk(X, y, l, beta):
    return l * (np.linalg.norm(beta)**2) + (1 / X.shape[0]) * (np.linalg.norm((X.dot(beta) - y) ** 2))

def ridge_regression_inv(X, y, l=1.0):
    n, d = X.shape
    a = (l * n) * np.identity(d) + np.transpose(X).dot(X)
    b = np.transpose(X).dot(y)
    beta = np.linalg.inv(a).dot(b)
    return beta

As the L2 norm is differentiable, learning problems using L2 regularization can also be solved by using gradient descent.

In [ ]:
def ridge_regression_grad(X, y, l=1, alpha=0.1):
    n, d = X.shape
    result = np.zeros((d, 1))
    cost_new = L2_regularized_risk(X, y, l, result)
    cost_old = 0
    i = 0
    while (cost_new - cost_old) ** 2 > 10 ** (-10):
        gradient = (np.transpose(X)).dot(X.dot(result) - y) + (l * n) * np.identity(d).dot(result) ## Gradient updated for the regularization.
        alpha = (np.transpose(gradient).dot(gradient)) / (
            (np.transpose(gradient).dot(np.transpose(X))).dot(X.dot(gradient)))
        result -= (2 * alpha / n) * gradient
        cost_new, cost_old = L2_regularized_risk(X, y, l, result), cost_new
        i += 1
    return result

### Standardization

The ridge solutions are not equivariant under scaling of the inputs, so one standardizes the inputs before solving the minimization problem. This means that we take

\begin{align}
x_{new} = \frac{x_{old}−x_{mean}}{x_{sd}}, \quad y_{new} = \frac{y_{old}−y_{mean}}{y_{sd}},
\end{align}

and arrange our learning algorithm accordingly.

In [ ]:
def ridge_regression_inv_std(X, y, l=1.0):
    ## Standardize data
    def standardize(X, mean_dataset, std_dataset):
        return (X-mean_dataset)/std_dataset
    
    x_mean = np.mean(X, axis=0)
    x_std = np.std(X, axis=0)
    y_mean = np.mean(y, axis=0)
    y_std = np.std(y, axis=0)
    
    X_new = standardize(X, x_mean, x_std)
    y_new = standardize(y, y_mean, y_std)
    
    ## The algorithm without standardization
    n, d = X_new.shape
    a = (l * n) * np.identity(d) + np.transpose(X_new).dot(X_new)
    b = np.transpose(X_new).dot(y_new)
    
    ## Renormalize beta
    beta_j = y_std * (np.linalg.inv(a).dot(b) / x_std.reshape(-1,1))
    beta_zero = y_mean - np.sum(beta_j*x_mean.reshape(-1,1))
    
    return np.vstack((beta_zero, beta_j))

In Scikit-learn, the **sklearn.linear_model** module has a built-in class for ridge regression with different computational solver possibilities.

In [ ]:
from sklearn.linear_model import Ridge

clf = Ridge(alpha=1.0, fit_intercept=True, normalize=True, solver='auto') 
            ## alpha corresponds to the regularization strength.

### Regularization hyperparameter

Note that the regularization coefficient $\lambda$ is a hyperparameter for the ridge regression. If $\lambda = 0$, we have the standard non-regularized linear regression model. On the other hand, setting $\lambda$ to a high value, the L2 norm of $\beta$ will be heavily constrained, leading to a simpler model that may underfit.

To see the effect of putting a penalty on the L2 norm, let us try to fit a high degree polynomial given a simple dataset. Note that the oscillations in the predictors are getting smaller and the models are becoming simpler for higher regularization coefficients.

In [ ]:
#np.random.seed(seed=781)
float_formatter = lambda x: "%.8f" % x
np.set_printoptions(formatter={'float_kind':float_formatter})

xp = np.linspace(-2, 2, 20)
yp = 10* xp ** 3 - 40 * xp + np.random.normal(0, 4, 20)
xp, yp = xp.reshape(-1,1), yp.reshape(-1,1)

from sklearn.preprocessing import PolynomialFeatures
def extend_sci(x, p): 
    poly = PolynomialFeatures(degree = p)
    return poly.fit_transform(x)

p=20 ## Degree of the polynomial.
a = np.linspace(min(xp),max(xp),num=1000).reshape(-1,1)
log_coef = np.linspace(-11, 1, 13) ## You may change the interval for zooming in!
coef = 10.0**log_coef 
for l in coef:
    plt.plot(xp, yp, 'ob')
    betap = ridge_regression_inv_std(extend_sci(xp, p)[:,1:], yp, l)
    b = extend_sci(a, p).dot(betap)
    plt.title('lambda = {}'.format(l))
    plt.ylim(-50,50)
    plt.plot(a, b, 'r')
    plt.show()

As another example, let us consider the Boston house-prices dataset.

In [ ]:
boston = datasets.load_boston()
x_raw, y_raw = boston.data, boston.target
y_raw = y_raw.reshape(-1,1)

We wil try to fit a third degree polynomial to the normalized dataset by using ridge regression. Note that the number of features in the extended array becomes greater than the number of available samples, which may lead to overfitting in the standard linear regression. We also need to remove the first column (containing 1s) after transforming our matrix since we need to exclude the intercept in our ridge algorithm.

In [ ]:
x_poly = extend_sci(x_raw, 3) 
x_poly = np.delete(x_poly, 0, 1) ## Removes the first column consisting of 1s.
print(x_poly.shape)

Let us prepare our data by shuffling and splitting into train, validation, and test sets.

In [ ]:
def shuffle(x, y):
    z = np.hstack((x, y))
    np.random.shuffle(z)
    return np.hsplit(z, [x.shape[1]])

x_poly, y = shuffle(x_poly, y_raw)

def validation_split(x, y, validation_size=0.15, test_size=0.15):
    n = x.shape[0]
    train_cut = int(np.floor(n * (1 - validation_size - test_size)))
    val_cut = train_cut + int(np.floor(n * validation_size))
    return x[:train_cut, ], x[train_cut:val_cut, ], x[val_cut:, ], y[:train_cut, ], y[train_cut:val_cut, ], y[val_cut:,]

x_train, x_validation, x_test, y_train, y_validation, y_test = validation_split(x_poly, y)

We may now use the validation set to compare the outcome for different values of $\lambda$ for hyperparameter selection.

In [ ]:
## Regularization coefficients in log-scale.
log_coef = np.linspace(-10, 5, 100) ## You may change the interval for zooming in!
coef = 10.0**log_coef 

reg_risk = []
reg_val_error = []
reg_R2 = []

def addones(x): ## Function for adding back the 1s we removed.
    x_add = np.hstack((np.ones((x.shape[0],1)), x))
    return x_add

for l in coef:
    beta = ridge_regression_inv_std(x_train, y_train, l) ## Parameters of our ridge regression algorithm.
        
    train_error = metrics.mean_squared_error(y_train, addones(x_train).dot(beta))
    reg_risk.append(train_error)
    
    val_error  = metrics.mean_squared_error(y_validation, addones(x_validation).dot(beta))
    reg_val_error.append(val_error)
    
    R2 = 1 - val_error / np.var(y_validation)
    reg_R2.append(R2)

    

plt.title('Training error')
plt.ylim(0,100)
plt.plot(log_coef, reg_risk)
plt.show()

plt.title('Least squares error on validation set')
plt.ylim(0,100)
plt.plot(log_coef, reg_val_error, 'r')
plt.show()

plt.title('R2 score on validation set')
plt.ylim(-1,1)
plt.plot(log_coef, reg_R2, color='darkred')
plt.show()


Selecting the hyperparameter that minimizes the least squares error on the validation set, we can test how our model performs. Note that the regularized version performs much better compared to the standard linear regression. 

In [ ]:
l_hyp = 10**(-1.5) ## Chosen from validation.

## Standard linear regression for comparison.
lin_reg = linear_model.LinearRegression(fit_intercept=True, normalize=False).fit(x_train, y_train)
beta_lin_reg = np.vstack((lin_reg.intercept_,lin_reg.coef_.reshape(-1,1)))
test_error_lin_reg = metrics.mean_squared_error(y_test, lin_reg.predict(x_test))

## Ridge regression with the chosen hyperparameter.
beta_ridge = ridge_regression_inv_std(x_train, y_train, l_hyp)
test_error_ridge = metrics.mean_squared_error(y_test, addones(x_test).dot(beta_ridge))

print("Test error without regularization:", test_error_lin_reg)
print("Test error with regularization:", test_error_ridge)

Let's also have a look at the parameters which were chosen:

In [ ]:
print("Difference of chosen parameters:", beta_lin_reg - beta_ridge)

In [ ]:
print("Parameter chosen by ridge:", beta_ridge)

A standard way of hyperparameter selection in this setting also is to use cross-validation techniques.

## L1 regularization - Lasso

Adding L1 regularization to a linear regression problem with the squared loss yields the *lasso* algorithm, defined as

$$ \arg \min_{β \in \mathbb{R}^d} \bigg( \lambda \| β \|_1  + \frac{1}{n} \sum_{i=1}^{n} \big(y_i - \langle β , x_i \rangle \big)^2 \bigg),$$

where we have the L1 norm $\| β \|_1 =\sum_{j=1}^{d} \vert β^{(j)} \vert$.

In practice, L1 regularization produces a sparse model that has most of its parameters $β^{(j)}$ equal to zero, provided the hyperparameter $\lambda$ is large enough. This leads to a feature selection by deciding which features are essential for prediction and which are not. 

Since the L1 penalty is not differentiable, different techniques from convex analysis and optimization, such as coordinate descent or subgradient methods, are required for obtaining solutions.

The lasso class in scikit-learn uses coordinate descent to fit the model.

In [ ]:
clf = linear_model.Lasso(alpha=0.0001, fit_intercept = True, normalize = True, max_iter=1000) 
                                                        ## More iterations may be needed for convergence
clf.fit(x_train, y_train)

beta_lasso = clf.coef_.reshape(-1,1)
print(beta_lasso)

Note that some of the parameters are set to zero in this case. Choosing a larger hyperparameter leads to fewer essential features and a simpler model.

In [ ]:
clf = linear_model.Lasso(alpha=0.01, fit_intercept = True, normalize=True)
clf.fit(x_train, y_train)

beta_lasso = clf.coef_.reshape(-1,1)
print(beta_lasso)

# Practice yourself!

1. Try to use k-fold (or leave-one-out) cross-validation for selecting the hyperparameter $\lambda$ in ridge regression for the Boston dataset.
2. Apply and compare the ridge regression and lasso on the diabetes dataset from scikit-learn. Try to observe the essential features from the lasso algorithm.
3. Try to implement the code for the coordinate descent in the lasso.
4. How would you apply L1 and L2 regularization together to linear regression with squared loss? What would be the advantages in this case? (Hint: search *elastic net regularization*.)